In [1]:
import sys
sys.path.append('../../Dataset-Management-Library/mimic3wfdb')
import json
import numpy as np
import pandas as pd
import psycopg2 as pg
import wfdb_util as wu
from itertools import starmap

In [2]:
with open('../../Dataset-Management-Library/pgconf.json') as f:
    pg_conf = json.load(f)

conn = pg.connect(
    host=pg_conf['host'], 
    port=pg_conf['port'], 
    dbname=pg_conf['dbname'], 
    user=pg_conf['user'], 
    password=pg_conf['password']
)

In [3]:
with conn.cursor() as cursor:
   cursor.execute(
      "SELECT *                   \
         FROM view_raw_demographic\
      ")

   result = cursor.fetchall()
cursor.close() # confirm

mimic_table = pd.DataFrame(result, columns=['subject_id', 'gender', 'age', 'admittime', 'dischtime', 'weight_kg', 'height_inches', 'height_cm', 'bmi'])
mimic_table = mimic_table.astype({'subject_id':'uint16', 'age':'uint8', 'weight_kg':'float32', 'height_inches':'float32', 'height_cm':'float32', 'bmi':'float32'})
display(mimic_table.head(3))
print(mimic_table.shape)

,subject_id,gender,age,admittime,dischtime,weight_kg,height_inches,height_cm,bmi
0,3,M,77,2101-10-20 19:08:00,2101-10-31 13:58:00,103.599998,70.5,179.100006,32.299999
1,9,M,42,2149-11-09 13:06:00,2149-11-14 10:15:00,102.699997,72.0,182.899994,30.700001
2,13,F,40,2167-01-08 18:43:00,2167-01-15 15:15:00,73.699997,57.0,144.800003,35.200001


(11871, 9)


In [4]:
record_list = wu.get_record_df('../../DataLake/mimic-iii_wfdb/')
record_list = record_list.query('record_name.str.contains("n")!=True').reset_index(drop=True)
record_list['recordtime'] = [pd.to_datetime(dt, format='%Y-%m-%d-%H-%M') for dt in map(lambda rec: rec[-1][8:-4], record_list.values)]
record_list['subject_id'] = record_list['subject_id'].astype(np.uint16)
display(record_list.head(3))
print(record_list.shape)
# duration: 10.7s

,subdir,subject_id,record_name,recordtime
0,p00,20,p000020-2183-04-28-17-47.hea,2183-04-28 17:47:00
1,p00,30,p000030-2172-10-16-12-22.hea,2172-10-16 12:22:00
2,p00,33,p000033-2116-12-24-12-35.hea,2116-12-24 12:35:00


(22317, 4)


In [5]:
mimic_join = pd.merge(left=mimic_table, right=record_list, how='inner', on=['subject_id'])
mimic_join = mimic_join.loc[(mimic_join['recordtime'] >= mimic_join['admittime'])&(mimic_join['recordtime'] < mimic_join['dischtime'])].sort_values(by=['subject_id', 'recordtime']).reset_index(drop=True)
mimic_join.sort_values(by='subject_id')

,subject_id,gender,age,admittime,dischtime,weight_kg,height_inches,height_cm,bmi,subdir,record_name,recordtime
0,20,F,76,2183-04-28 09:45:00,2183-05-03 14:45:00,93.300003,65.0,165.100006,34.200001,p00,p000020-2183-04-28-17-47.hea,2183-04-28 17:47:00
1,30,M,90,2172-10-14 14:17:00,2172-10-19 14:37:00,65.699997,65.0,165.100006,24.100000,p00,p000030-2172-10-16-12-22.hea,2172-10-16 12:22:00
2,79,M,52,2175-09-25 23:05:00,2175-09-29 14:10:00,77.199997,70.0,177.800003,24.400000,p00,p000079-2175-09-26-01-25.hea,2175-09-26 01:25:00
3,79,M,52,2175-09-25 23:05:00,2175-09-29 14:10:00,77.199997,70.0,177.800003,24.400000,p00,p000079-2175-09-26-12-28.hea,2175-09-26 12:28:00
7,124,M,70,2160-06-24 21:25:00,2160-07-15 15:10:00,68.000000,69.0,175.300003,22.100000,p00,p000124-2160-07-11-12-21.hea,2160-07-11 12:21:00
...,...,...,...,...,...,...,...,...,...,...,...,...
2439,32805,M,66,2182-11-14 11:37:00,2182-12-26 14:55:00,88.900002,72.0,182.899994,26.600000,p03,p032805-2182-12-20-13-02.hea,2182-12-20 13:02:00
2435,32805,M,66,2182-11-14 11:37:00,2182-12-26 14:55:00,88.900002,72.0,182.899994,26.600000,p03,p032805-2182-12-11-09-32.hea,2182-12-11 09:32:00
2434,32805,M,66,2182-11-14 11:37:00,2182-12-26 14:55:00,88.900002,72.0,182.899994,26.600000,p03,p032805-2182-12-10-17-19.hea,2182-12-10 17:19:00
2436,32805,M,66,2182-11-14 11:37:00,2182-12-26 14:55:00,88.900002,72.0,182.899994,26.600000,p03,p032805-2182-12-11-10-46.hea,2182-12-11 10:46:00


In [7]:
mimic_join.to_parquet(path='../data/intersectDemoWave.parquet.gzip', engine='pyarrow', compression='gzip')